# Machine Learning Model

> In this module, we develop trainers of different models

In [1]:
#| default_exp train

In [2]:
#| hide
import sys
sys.path.append("/notebooks/katlas")
from nbdev.showdoc import *
%matplotlib inline
%load_ext autoreload
%autoreload 2


In [3]:
#| export
from katlas.core import Data
from katlas.feature import *
from fastbook import *
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.stats import spearmanr,pearsonr
from sklearn.model_selection import train_test_split
from pathlib import Path
from sklearn.metrics import mean_squared_error
import math
from scipy.stats import spearmanr, pearsonr
import joblib
from sklearn.linear_model import *
from sklearn.svm import *
from sklearn.ensemble import *

## Prepare dataset

In [4]:
df = Data.get_prepared_data()

In [5]:
df

,kinase,substrate,target,position,aa,esm_PCA1,esm_PCA2,esm_PCA3,esm_PCA4,esm_PCA5,esm_PCA6,esm_PCA7,esm_PCA8,esm_PCA9,esm_PCA10,esm_PCA11,esm_PCA12,esm_PCA13,esm_PCA14,esm_PCA15,esm_PCA16,esm_PCA17,esm_PCA18,esm_PCA19,esm_PCA20,esm_PCA21,esm_PCA22,esm_PCA23,esm_PCA24,esm_PCA25,esm_PCA26,esm_PCA27,esm_PCA28,esm_PCA29,esm_PCA30,esm_PCA31,esm_PCA32,esm_PCA33,esm_PCA34,esm_PCA35,esm_PCA36,esm_PCA37,esm_PCA38,esm_PCA39,esm_PCA40,esm_PCA41,esm_PCA42,esm_PCA43,esm_PCA44,esm_PCA45,esm_PCA46,esm_PCA47,esm_PCA48,esm_PCA49,esm_PCA50,esm_PCA51,esm_PCA52,esm_PCA53,esm_PCA54,esm_PCA55,esm_PCA56,esm_PCA57,esm_PCA58,esm_PCA59,esm_PCA60,esm_PCA61,esm_PCA62,esm_PCA63,esm_PCA64,aa_PCA1,aa_PCA2,aa_PCA3,aa_PCA4,aa_PCA5,aa_PCA6,aa_PCA7,aa_PCA8,aa_PCA9,aa_PCA10,aa_PCA11,aa_PCA12,aa_PCA13,aa_PCA14,aa_PCA15,aa_PCA16
0,AAK1,-5P,1.667971,-5,P,-0.216769,-0.733580,-0.344105,-0.128011,-0.132599,0.291402,0.276428,0.436074,0.082743,-0.154011,0.160621,-0.139237,-0.24736,-0.063899,0.057053,-0.150866,0.057690,-0.122848,0.142676,-0.221692,0.235782,-0.045391,0.356115,0.054616,-0.046005,-0.026163,0.052584,-0.076907,-0.087619,-0.107619,0.140290,0.054464,0.019788,0.045404,-0.00397,-0.031520,0.094496,-0.035393,0.033353,0.004383,-0.041130,0.063835,-0.032990,-0.163464,-0.028508,0.097613,0.066614,-0.153882,-0.141769,-0.035814,-0.028078,-0.092820,-0.017141,0.001073,-0.070159,-0.005157,-0.048426,0.036052,-0.157780,0.086889,-0.083062,-0.022601,-0.075341,0.056845,-4.604141,2.659866,1.697217,-0.613948,1.642941,-1.759448,4.083249,-1.740442,3.579492,3.997759,1.403490,0.224282,-0.163520,-0.649079,-0.057365,0.004034
1,AAK1,-4P,1.152337,-4,P,-0.216769,-0.733580,-0.344105,-0.128011,-0.132599,0.291402,0.276428,0.436074,0.082743,-0.154011,0.160621,-0.139237,-0.24736,-0.063899,0.057053,-0.150866,0.057690,-0.122848,0.142676,-0.221692,0.235782,-0.045391,0.356115,0.054616,-0.046005,-0.026163,0.052584,-0.076907,-0.087619,-0.107619,0.140290,0.054464,0.019788,0.045404,-0.00397,-0.031520,0.094496,-0.035393,0.033353,0.004383,-0.041130,0.063835,-0.032990,-0.163464,-0.028508,0.097613,0.066614,-0.153882,-0.141769,-0.035814,-0.028078,-0.092820,-0.017141,0.001073,-0.070159,-0.005157,-0.048426,0.036052,-0.157780,0.086889,-0.083062,-0.022601,-0.075341,0.056845,-4.604141,2.659866,1.697217,-0.613948,1.642941,-1.759448,4.083249,-1.740442,3.579492,3.997759,1.403490,0.224282,-0.163520,-0.649079,-0.057365,0.004034
2,AAK1,-3P,1.954319,-3,P,-0.216769,-0.733580,-0.344105,-0.128011,-0.132599,0.291402,0.276428,0.436074,0.082743,-0.154011,0.160621,-0.139237,-0.24736,-0.063899,0.057053,-0.150866,0.057690,-0.122848,0.142676,-0.221692,0.235782,-0.045391,0.356115,0.054616,-0.046005,-0.026163,0.052584,-0.076907,-0.087619,-0.107619,0.140290,0.054464,0.019788,0.045404,-0.00397,-0.031520,0.094496,-0.035393,0.033353,0.004383,-0.041130,0.063835,-0.032990,-0.163464,-0.028508,0.097613,0.066614,-0.153882,-0.141769,-0.035814,-0.028078,-0.092820,-0.017141,0.001073,-0.070159,-0.005157,-0.048426,0.036052,-0.157780,0.086889,-0.083062,-0.022601,-0.075341,0.056845,-4.604141,2.659866,1.697217,-0.613948,1.642941,-1.759448,4.083249,-1.740442,3.579492,3.997759,1.403490,0.224282,-0.163520,-0.649079,-0.057365,0.004034
3,AAK1,-2P,0.485449,-2,P,-0.216769,-0.733580,-0.344105,-0.128011,-0.132599,0.291402,0.276428,0.436074,0.082743,-0.154011,0.160621,-0.139237,-0.24736,-0.063899,0.057053,-0.150866,0.057690,-0.122848,0.142676,-0.221692,0.235782,-0.045391,0.356115,0.054616,-0.046005,-0.026163,0.052584,-0.076907,-0.087619,-0.107619,0.140290,0.054464,0.019788,0.045404,-0.00397,-0.031520,0.094496,-0.035393,0.033353,0.004383,-0.041130,0.063835,-0.032990,-0.163464,-0.028508,0.097613,0.066614,-0.153882,-0.141769,-0.035814,-0.028078,-0.092820,-0.017141,0.001073,-0.070159,-0.005157,-0.048426,0.036052,-0.157780,0.086889,-0.083062,-0.022601,-0.075341,0.056845,-4.604141,2.659866,1.697217,-0.613948,1.642941,-1.759448,4.083249,-1.740442,3.579492,3.997759,1.403490,0.224282,-0.163520,-0.649079,-0.057365,0.004034
4,AAK1,-1P,2.791637,-1,P,-0.216769,-0.733580,

In [6]:
feat_col = ['position'] + df.columns.tolist()[5:]

target_col = ['target']

X = df[feat_col]
y = df[target_col]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)


In [7]:
X_train

,position,esm_PCA1,esm_PCA2,esm_PCA3,esm_PCA4,esm_PCA5,esm_PCA6,esm_PCA7,esm_PCA8,esm_PCA9,esm_PCA10,esm_PCA11,esm_PCA12,esm_PCA13,esm_PCA14,esm_PCA15,esm_PCA16,esm_PCA17,esm_PCA18,esm_PCA19,esm_PCA20,esm_PCA21,esm_PCA22,esm_PCA23,esm_PCA24,esm_PCA25,esm_PCA26,esm_PCA27,esm_PCA28,esm_PCA29,esm_PCA30,esm_PCA31,esm_PCA32,esm_PCA33,esm_PCA34,esm_PCA35,esm_PCA36,esm_PCA37,esm_PCA38,esm_PCA39,esm_PCA40,esm_PCA41,esm_PCA42,esm_PCA43,esm_PCA44,esm_PCA45,esm_PCA46,esm_PCA47,esm_PCA48,esm_PCA49,esm_PCA50,esm_PCA51,esm_PCA52,esm_PCA53,esm_PCA54,esm_PCA55,esm_PCA56,esm_PCA57,esm_PCA58,esm_PCA59,esm_PCA60,esm_PCA61,esm_PCA62,esm_PCA63,esm_PCA64,aa_PCA1,aa_PCA2,aa_PCA3,aa_PCA4,aa_PCA5,aa_PCA6,aa_PCA7,aa_PCA8,aa_PCA9,aa_PCA10,aa_PCA11,aa_PCA12,aa_PCA13,aa_PCA14,aa_PCA15,aa_PCA16
14144,1,-1.369722,0.197498,0.223170,0.040395,-0.189029,-0.224488,0.045137,0.150561,-0.134846,0.087944,-0.223046,-0.060643,0.138707,-0.118685,-0.035282,-0.015771,-0.070491,-0.234602,-0.165760,0.105652,-0.038383,0.085017,-0.088635,0.020698,-0.053518,0.030420,-0.007120,-0.069412,-0.018899,0.143355,0.050146,-0.065123,-0.003086,-0.137633,0.090300,0.016716,-0.054610,-0.047686,0.021627,-0.015834,0.070976,0.018288,-0.067827,-0.015008,0.013781,-0.035464,0.008310,0.000575,-0.068981,-0.056833,-0.086908,0.050954,-0.012423,-0.017409,-0.017336,0.005927,-0.068158,0.049561,-0.010132,-0.035070,0.118808,0.022446,-0.004756,0.057267,-0.764338,-3.284465,-0.883925,-0.483551,0.807965,-3.361002,2.279310,0.075253,-2.333592,-1.376925,1.089510,1.278021,-0.903551,0.681257,-0.611565,0.542652
28098,-5,-0.748237,0.236742,-0.767538,0.017594,0.429834,-0.008406,-0.073802,-0.018438,0.143914,0.032871,-0.055416,0.174181,0.000455,-0.084757,0.372829,-0.062349,-0.114727,-0.018027,0.025977,0.130442,0.003969,-0.132158,-0.192100,0.115313,0.051776,-0.109625,-0.113333,0.080498,0.057037,-0.002202,-0.062761,-0.043099,0.140517,0.053927,-0.012054,-0.027907,0.022691,-0.078691,-0.020139,-0.006611,0.025090,-0.016265,0.083504,-0.062135,0.032219,-0.006140,-0.129990,-0.024461,0.008433,0.045387,0.058840,0.028120,-0.011980,0.080835,-0.017266,0.014087,0.027079,0.020167,-0.131479,0.142231,0.075436,-0.001224,-0.059097,-0.075540,-0.630406,4.541536,-2.464032,-2.528216,-0.388043,0.554349,-1.136118,-0.456329,-2.472756,0.709053,0.354624,-0.836632,-2.015800,-1.683884,1.464726,-0.859887
20991,-2,-2.255517,0.870975,-0.161415,0.498748,0.110903,-0.059210,0.017561,-0.052159,-0.215684,-0.286370,-0.246190,0.157692,0.000618,-0.264107,-0.010539,0.003370,-0.096015,-0.211218,0.009766,-0.008676,-0.089885,0.081214,0.107962,-0.041321,0.196832,0.191719,0.045814,-0.225606,-0.023087,-0.070228,-0.019106,0.059450,-0.073517,0.030298,-0.219100,-0.095247,0.047371,0.168666,0.173706,0.006751,-0.040738,-0.001904,-0.072111,0.075768,-0.055271,0.126734,-0.040563,-0.018217,0.050050,-0.130689,-0.008719,-0.034561,-0.012439,-0.108195,0.066205,-0.094110,-0.036702,-0.057620,-0.062445,-0.059649,-0.041827,-0.000388,0.043506,-0.162113,-2.426990,1.163650,-4.728877,0.464861,1.585046,-1.912505,-1.424945,1.208934,1.249052,0.311335,-0.671125,0.504825,-0.239695,0.184534,-0.137854,-1.100433
8581,-1,1.327566,0.323383,-0.299326,-0.055454,-0.464042,0.118666,-0.334923,-0.010333,-0.190573,-0.175225,0.004014,0.192834,-0.023488,0.268127,0.109572,0.099034,-0.001119,0.137899,-0.000330,0.131663,0.061472,-0.197859,0.028100,0.004128,-0.131371,0.022670,-0.042120,-0.075315,-0.067843,0.041057,-0.005008,-0.057124,0.036451,-0.169856,-0.112805,-0.051747,-0.089930,-0.096612,-0.020140,-0.104599,0.008080,-0.109656,-0.000237,0.024323,-0.074875,-0.048211,-0.085576,-0.103952,0.089011,-0.055950,-0.025368,-0.082950,0.052899,-0.035035,0.024958,-0.176769,-0.050345,0.057378,-0.014163,0.043186,0.080827,0.062216,0.031022,-0.073623,-3.213352,-1.699523,3.189200,-0.265951,1.811946,2.088905,-1.583068,-0.483111,1.266202,-0.753519,-0.837779,3.376496,-0.407172,1.326932,2.198692,-0.446317
52181,4,-1.786632,0.209399,-0.911660,0.307268,0.218209,-0.053392,0.387789,0.021070,0.071548,-0.149715,-0.099283,0.250731,-0.2

## sklearn ML regressors

In [8]:
#| export
def train_ML(model, 
             X_train, 
             X_test, 
             y_train, 
             y_test, 
             save = None, # file name (.pkl) to save
            ):
    
    # Fit the model
    model.fit(X_train, y_train.values.ravel()) # better convert y_train to numpy array and flatten
    print(model)
    
    if save is not None:
        # Save the model to a file
        joblib.dump(model, save)
    
    # Predict
    y_pred = model.predict(X_test) # X_test is dataframe, y_pred is numpy array
    
    # Prepare pred dataframe
    pred = y_test.copy()
    pred['pred'] = y_pred
    
    # Calculate RMSE
    mse = mean_squared_error(pred.iloc[:,0], pred.pred)
    rmse = math.sqrt(mse)
    print(f'rmse is {rmse:.4f}')
    
    # Calculate the Spearman correlation coefficient
    spearman_corr, _ = spearmanr(pred.iloc[:,0], pred.pred)

    # Calculate the Pearson correlation coefficient
    pearson_corr, _ = pearsonr(pred.iloc[:,0], pred.pred)

    print(f"Spearman correlation coefficient: {spearman_corr:.4f}")
    print(f"Pearson correlation coefficient: {pearson_corr:.4f} ")
    
    return pred

### LinearRegression

without any regularization

In [9]:
model = LinearRegression()

pred = train_ML(model,X_train, X_test, y_train, y_test)

LinearRegression()
rmse is 0.9592
Spearman correlation coefficient: 0.3173
Pearson correlation coefficient: 0.2805 


## XGB

In [10]:
#| export
def xgb_trainer(df,
                feature_col,
                target_col,
                test_index=None,
                xgb_params = { 
                            'max_depth':7, #from 4 to 7
                            'learning_rate':0.001, #from 0.001
                            'subsample':0.8,
                            'colsample_bytree':1, # from 0.2 to 1, because need to take all features
                            'eval_metric':'rmse',
                            'objective':'reg:squarederror',
                            'tree_method':'gpu_hist',
                            'predictor':'gpu_predictor',
                            'random_state':123
                        },
                model_file='xgb_model.bin',
                split_seed = 123, # seed of random split
               ):
    
    X = df[feature_col]
    y = df[target_col]
    
    print(f'xgb params is: {xgb_params}')
    
    if test_index is None:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=split_seed)
    else:
        X_train,y_train = X.loc[~X.index.isin(test_index)],y.loc[~X.index.isin(test_index)]
        X_test, y_test = X.loc[test_index],y.loc[test_index]

        
    print(X_train.shape,y_train.shape,X_test.shape, y_test.shape)
    print(y_test.index)
    #prepare matrix for xgb
    dtrain = xgb.DMatrix(X_train, y_train)
    dtest = xgb.DMatrix(X_test, y_test)
    
    model = xgb.train(xgb_params, 
            dtrain=dtrain,
            evals=[(dtrain,'train'),(dtest,'valid')],
            num_boost_round=9999,
            early_stopping_rounds=100,
            verbose_eval=100,)
    
    # Save the model
    path = Path(model_file)
    
    # Make a directory if not exists
    path.parent.mkdir(exist_ok=True)
        
    model.save_model(model_file)
    print(f'Model saved to {model_file}')
    
    # Prepare the pred/target df
    pred = model.predict(dtest)
    
    out = np.vstack([np.ravel(y_test),np.ravel(pred)]).T
    pred_df = pd.DataFrame(out,index=y_test.index, columns = ['target','pred'] )
    
    spearman_corr, _ = spearmanr(pred_df.target, pred_df.pred)
    print(f'Spearman correlation: {spearman_corr:.2f}')
    pearson_corr, p_value = pearsonr(pred_df.target, pred_df.pred)
    print(f'Pearson correlation: {pearson_corr:.2f}')


    
    fig, ax = plt.subplots()
    ax.scatter(pred_df.target, pred_df.pred)
    ax.set_xlabel('True values')
    ax.set_ylabel('Predicted values')
    ax.set_title('Scatter plot of true versus predicted values')
    plt.show()
    plt.close()
    
    
    dd = model.get_score(importance_type='gain')
    gain = pd.DataFrame({'feature':dd.keys(),f'gain_importance':dd.values()}).set_index('feature').sort_values(by='gain_importance',ascending=False)
    gain[:10].plot.barh()
    plt.show()
    plt.close()
        
    dd = model.get_score(importance_type='weight')
    weight = pd.DataFrame({'feature':dd.keys(),f'weight_importance':dd.values()}).set_index('feature').sort_values(by='weight_importance',ascending=False)
    weight[:10].plot.barh()
    plt.show()
    plt.close()
    
    return pred_df, gain, weight

In [11]:
show_doc(xgb_trainer)

---

### xgb_trainer

>      xgb_trainer (df, feature_col, target_col, test_index=None,
>                   xgb_params={'max_depth': 7, 'learning_rate': 0.001,
>                   'subsample': 0.8, 'colsample_bytree': 1, 'eval_metric':
>                   'rmse', 'objective': 'reg:squarederror', 'tree_method':
>                   'gpu_hist', 'predictor': 'gpu_predictor', 'random_state':
>                   123}, model_file='xgb_model.bin', split_seed=123)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df |  |  |  |
| feature_col |  |  |  |
| target_col |  |  |  |
| test_index | NoneType | None |  |
| xgb_params | dict | {'max_depth': 7, 'learning_rate': 0.001, 'subsample': 0.8, 'colsample_bytree': 1, 'eval_metric': 'rmse', 'objective': 'reg:squarederror', 'tree_method': 'gpu_hist', 'predictor': 'gpu_predictor', 'random_state': 123} |  |
| model_file | str | xgb_model.bin |  |
| split_seed | int | 123 | seed of random split |

In [12]:
# xgb_trainer(df,feat_col, target_col)

In [13]:
#| export
def xgb_predict(df, # a dataframe that contains ID and features for prediction
                feature_col, #feature column name
                ID_col = "ID", #ID column name
                model_file='xgb_model.bin'):
    # Load the XGBoost model
    model = xgb.Booster()
    model.load_model(model_file)
    
    # Prepare data for prediction
    X = df[feature_col]
    dtest = xgb.DMatrix(X)
    
    # Make predictions
    preds = model.predict(dtest)
    
    # Combine predictions with IDs into a DataFrame
    result_df = pd.DataFrame({ID_col: df[ID_col], 'preds': preds})
    
    return result_df

In [14]:
show_doc(xgb_predict)

---

### xgb_predict

>      xgb_predict (df, feature_col, ID_col='ID', model_file='xgb_model.bin')

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df |  |  | a dataframe that contains ID and features for prediction |
| feature_col |  |  | feature column name |
| ID_col | str | ID | ID column name |
| model_file | str | xgb_model.bin |  |

In [15]:
#| hide
import nbdev; nbdev.nbdev_export()